In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV

Load Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/code/scikit-learn-sentiment-analysis/data/movie_data.csv')

term frequency-inverse document frequency

In [ ]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

prep data

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [ ]:
df['review'] = df["review"].apply(preprocessor)

Stem & Tokenize Documents & remove Stopwords

In [ ]:
porter = PorterStemmer()

In [ ]:
def tokenizer (text):
  return[porter.stem(word) for word in text.split()]

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

tokens to tf-idf Vectors

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                         lowercase=False,
                         preprocessor=None,
                         tokenizer=tokenizer,
                         use_idf=True,
                         norm='l2',
                         smooth_idf=True)

y = df.sentiment.values
X = tfidf.fit_transform(df.review)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Logistical Regression Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)

In [ ]:
clf = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=0,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X_train, y_train)
saved_model = open('/content/drive/MyDrive/code/scikit-learn-sentiment-analysis/saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.8min finished


accuracy

In [ ]:
filename = '/content/drive/MyDrive/code/scikit-learn-sentiment-analysis/saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [ ]:
saved_clf.score(X_test, y_test)

0.89608